In [83]:
# Imports
import os
from collections import Counter
from PyPDF2 import PdfReader
import pdfminer
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [79]:
labs = ['ACT Laboratories', 'Lexachrom Analytical Laboratory', 'Talon Analytical', 'Phyto-Farma Labs', 'Biotrax Testing Laboratory', 'Kaycha Labs', 'Keystone State Testing', 'MCR', 'Certified Testing and Data', 'Botannis Labs', 'Reliable Labs', 'Certainty Analytical Labs', 'Green Analytics', 'Coral Reef Labs', 'SC Laboratories', 'other']
brands = ['Flower House']
dispensaries = ['Gotham']
weed_frame = pd.DataFrame(columns=['name', 'brand', 'dispensary', 'lab'])

def input_strains(df: pd.DataFrame, dispensary: str, brand: str) -> None:
  directory = f"lab_pdf_dump/{dispensary}/{brand}"
  for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
      filepath = os.path.join(directory, filename)
      name = filename.removesuffix('.pdf')
      reader = PdfReader(filepath)
      text = reader.pages[0].extract_text()
      has_lab = False
      
      for lab in labs:
        if lab.lower() in text.lower():
          df.loc[len(df.index)] = [name, brand, dispensary, lab]
          has_lab = True
          break
          
      if not has_lab:
        df.loc[len(df.index)] = [name, brand, dispensary, 'other']
        
  df.drop_duplicates(inplace=True)

In [81]:
# XML Analysis
input_strains(weed_frame, 'Gotham', 'Flower House')
weed_frame

,name,brand,dispensary,lab
0,FH0219 Oreoz 0.5g Pre-rolls,Flower House,Gotham,Keystone State Testing
1,FH0353 Gelato 3.5g Flower,Flower House,Gotham,Keystone State Testing
2,FH0375 White Cherry Gelato 0.5g Preroll 7pk,Flower House,Gotham,Keystone State Testing
3,FH0364 Mendo Breath 1g Flower,Flower House,Gotham,Keystone State Testing
4,FH0363 Gelato 1g Flower,Flower House,Gotham,Keystone State Testing
...,...,...,...,...
82,FH0325 White Cherry Gelato 3.5g Flower,Flower House,Gotham,Kaycha Labs
83,FH0310 Italian Ice 3.5g Flower,Flower House,Gotham,Kaycha Labs
84,FH0303 Bigfoot Glue 0.5g Pre-rolls,Flower House,Gotham,Keystone State Testing
85,FH0221 Gelato 0.5g Pre-rolls,Flower House,Gotham,Keystone State Testing


In [85]:
# PDF Downloader
# Base URL
base_url = "https://www.mycoa.info"  # replace with your target url

# Send HTTP request
response = requests.get(base_url)

# Parse HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all links and filter on .pdf extension
pdf_links = [urljoin(base_url, link['href']).replace('dl=0', 'dl=1') for link in soup.find_all('a') if 'dropbox.com' in link['href']]

# Directory to store the downloaded pdf files
download_dir = "pdf_downloads"

# Ensure download directory exists
os.makedirs(download_dir, exist_ok=True)

for link in pdf_links:
    # Send HTTP request
    file_response = requests.get(link, stream=True)
    
    # Get file name
    file_name = os.path.join(download_dir, link.split("/")[-1].split('?')[0])

    # Write content to file
    with open(file_name, 'wb') as file:
        for chunk in file_response.iter_content(chunk_size=1024): 
            if chunk:
                file.write(chunk)

print(f"Downloaded {len(pdf_links)} pdf files.")


Downloaded 41 pdf files.
